### This script is the extension of previous script replacing Linear_Classifier with DNNClassifier

### Setup

In [1]:
import glob
import io
import os
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import style
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = ":.1f".format
style.use('ggplot')

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load the Dataset and Randomization

In [2]:
mnist_dataframe = pd.read_csv(
    io.open('mnist_train_small.csv'),sep=",",
    header=None)
mnist_dataframe = mnist_dataframe.head(10000)
mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))
mnist_dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
3557,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3259,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6135,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8890,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
display.display(mnist_dataframe.loc[2:3, 10:500])
display.display(mnist_dataframe[0]) # labels
display.display(mnist_dataframe.loc[:, 1:784]) # features

,10,11,12,13,14,15,16,17,18,19,...,491,492,493,494,495,496,497,498,499,500
2,0,0,0,0,0,0,0,0,0,0,...,0,2,129,253,253,15,0,0,0,0
1121,0,0,0,0,0,0,0,0,0,0,...,206,246,254,240,210,228,254,229,32,0
2428,0,0,0,0,0,0,0,0,0,0,...,208,0,0,0,0,0,0,0,0,0
6525,0,0,0,0,0,0,0,0,0,0,...,211,171,171,171,171,176,254,254,254,254
633,0,0,0,0,0,0,0,0,0,0,...,253,114,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0,0,0,0,0,0,0,0,0,0,...,253,253,253,253,253,253,253,83,0,0
6413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,51,252,253,50,0,0,0
6254,0,0,0,0,0,0,0,0,0,0,...,253,176,0,0,0,0,42,245,253,35
1056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,37,253,155,0,0


3557    3
3259    2
6135    1
8445    0
8890    2
       ..
9055    2
7307    3
1647    2
5553    1
2014    3
Name: 0, Length: 10000, dtype: int64

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
3557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5553,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Parse out Labels and Features

In [11]:
def parse_labels_and_features(dataset):
    labels = dataset[0]
    features = dataset[:, 1:784]
    features = features / 255 # gray scale
    return labels, features

### Training set and Validation set